<a href="https://colab.research.google.com/github/seyoung911/ClearWater/blob/master/Assignment11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import numpy as np
import re
import nltk
from sklearn.datasets import load_files
nltk.download('stopwords')
nltk.download('wordnet')
import pickle
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split


review_data = load_files("/content/drive/My Drive/Colab Notebooks/movie_review")
X, y = review_data.data, review_data.target

documents = []

stemmer = WordNetLemmatizer()

for sen in range(0, len(X)):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(X[sen]))
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', ' ', document)
    
    # Converting to Lowercase
    document = document.lower()
    
    # Lemmatization
    document = document.split()
    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    
    documents.append(document)

vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X = vectorizer.fit_transform(documents).toarray()

tfidfconverter = TfidfTransformer()
X = tfidfconverter.fit_transform(X).toarray()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)





In [134]:

import numpy as np
import random

def sigmoid(x, Derivative = False):
  if not Derivative:
    return 1 / (1 + np.exp(-x))
  else:
    out = sigmoid(x)
    return out * (1 - out)

class backPropNN:

  # class Members( iternal variables that are accessed with backPropNN.member)

  numLayers = 0
  shape = None
  weights = []

  def __init__(self, numNodes):

    """Initialize the NN - setup the layers and initial weights"""
    self.numLayers = len(numNodes) - 1
    self.shape = numNodes

    for (i, j) in zip(numNodes[:-1], numNodes[1:]):
      self.weights.append(np.random.normal( 0.1, (j, i+1)))

    self._layerInput = []
    self._layerOutput = []

  def FP(self, input):
    """Get the input data and run it through the NN"""

    self._layerInput = []
    self._layerOutput = []

    numExamples = input.shape[0] #input 행렬의 행 갯수. = 예시 데이터 갯수(len(X))

    for index in range(self.numLayers):
      if (index == 0) :
        layerInput = self.weights[0].dot(np.vstack([input.T, np.ones([1, numExamples])]))
      else :
        layerInput = self.weights[index].dot(np.vstack([self._layerOutput[-1], np.ones([1, numExamples])]))
        
      self._layerInput.append(layerInput)
      self._layerOutput.append(sigmoid(layerInput))

    return self._layerOutput[-1].T

  def backProp(self, input, target, trainingRate = 0.2):

    """Get the error, deltas and back propagate to update the weights"""

    delta = []
    numExamples = input.shape[0]

    self.FP(input)
    
    for index in reversed(range(self.numLayers)):

      if index == self.numLayers -1:
        output_delta = self._layerOutput[index] - target.T
        error = np.sum(ouput_delta ** 2)
        delta.append(output_delta * sigmoid(self._layerInput[index],True))

      else:
        delta_pullback = self.weights[index +1].T.dot(delta[-1])
        delta.append(delta_pullback[:-1, :] * sigmoid(self._layerInput[index],True))
  
    for index in range(self.numLayers):
      delta_index = self.numLayers -1-index

      if index == 0:
        layerOutput = np.vstack([input.T, np.ones([1, numExamples])])
      else:
        layerOutput = np.vstack([self._layerOutput[index - 1], np.ones([1,self._layerOutput[index -1].shape[1]])])
       
        return error


NN = backPropNN((1400,50,1))
Error = NN.backProp(X_train, y_train)
Output = NN.FP(X_train)


ValueError: ignored

In [ ]:
# 이 코드 이용해 plot 시도!

nn_architecture = [
    {"input_dim": 1500, "output_dim": 700, "activation": "relu"},
    {"input_dim": 700, "output_dim": 300, "activation": "relu"},
    {"input_dim": 300, "output_dim": 100, "activation": "relu"},
    {"input_dim": 100, "output_dim": 20, "activation": "relu"},
    {"input_dim": 20, "output_dim": 1, "activation": "sigmoid"},
]

def init_layers(nn_architecture, seed = 99):
    np.random.seed(seed)
    number_of_layers = len(nn_architecture)
    params_values = {}

    for idx, layer in enumerate(nn_architecture):
        layer_idx = idx + 1
        layer_input_size = layer["input_dim"]
        layer_output_size = layer["output_dim"]
        
        params_values['W' + str(layer_idx)] = np.random.randn(
            layer_output_size, layer_input_size) * 0.1
        params_values['b' + str(layer_idx)] = np.random.randn(
            layer_output_size, 1) * 0.1
        
    return params_values

def sigmoid(Z):
    return 1/(1+np.exp(-Z))

def relu(Z):
    return np.maximum(0,Z)

def sigmoid_backward(dA, Z):
    sig = sigmoid(Z)
    return dA * sig * (1 - sig)

def relu_backward(dA, Z):
    dZ = np.array(dA, copy = True)
    dZ[Z <= 0] = 0;
    return dZ;


def single_layer_forward_propagation(A_prev, W_curr, b_curr, activation="relu"):

    Z_curr = np.dot(W_curr, A_prev) + b_curr
    
    if activation is "relu":
        activation_func = relu
    elif activation is "sigmoid":
        activation_func = sigmoid
    else:
        raise Exception('Non-supported activation function')
        
    return activation_func(Z_curr), Z_curr

def full_forward_propagation(X, params_values, nn_architecture):
    memory = {}
    A_curr = X
    
    for idx, layer in enumerate(nn_architecture):
        layer_idx = idx + 1
        A_prev = A_curr
        
        activ_function_curr = layer["activation"]
        W_curr = params_values["W" + str(layer_idx)]
        b_curr = params_values["b" + str(layer_idx)]
        A_curr, Z_curr = single_layer_forward_propagation(A_prev, W_curr, b_curr, activ_function_curr)
        
        memory["A" + str(idx)] = A_prev
        memory["Z" + str(layer_idx)] = Z_curr
       
    return A_curr, memory

def get_cost_value(Y_hat, Y):
    m = Y_hat.shape[1]
    cost = -1 / m * (np.dot(Y, np.log(Y_hat).T) + np.dot(1 - Y, np.log(1 - Y_hat).T))
    return np.squeeze(cost)

def get_accuracy_value(Y_hat, Y):
    Y_hat_ = convert_prob_into_class(Y_hat)
    return (Y_hat_ == Y).all(axis=0).mean()

def single_layer_backward_propagation(dA_curr, W_curr, b_curr, Z_curr, A_prev, activation="relu"):
    m = A_prev.shape[1]
    
    if activation is "relu":
        backward_activation_func = relu_backward
    elif activation is "sigmoid":
        backward_activation_func = sigmoid_backward
    else:
        raise Exception('Non-supported activation function')
    
    dZ_curr = backward_activation_func(dA_curr, Z_curr)
    dW_curr = np.dot(dZ_curr, A_prev.T) / m
    db_curr = np.sum(dZ_curr, axis=1, keepdims=True) / m
    dA_prev = np.dot(W_curr.T, dZ_curr)

    return dA_prev, dW_curr, db_curr


def full_backward_propagation(Y_hat, Y, memory, params_values, nn_architecture):
    grads_values = {}
    m = Y.shape[1]
    Y = Y.reshape(Y_hat.shape)
   
    dA_prev = - (np.divide(Y, Y_hat) - np.divide(1 - Y, 1 - Y_hat));
    
    for layer_idx_prev, layer in reversed(list(enumerate(nn_architecture))):
        layer_idx_curr = layer_idx_prev + 1
        activ_function_curr = layer["activation"]
        
        dA_curr = dA_prev
        
        A_prev = memory["A" + str(layer_idx_prev)]
        Z_curr = memory["Z" + str(layer_idx_curr)]
        W_curr = params_values["W" + str(layer_idx_curr)]
        b_curr = params_values["b" + str(layer_idx_curr)]
        
        dA_prev, dW_curr, db_curr = single_layer_backward_propagation(
            dA_curr, W_curr, b_curr, Z_curr, A_prev, activ_function_curr)
        
        grads_values["dW" + str(layer_idx_curr)] = dW_curr
        grads_values["db" + str(layer_idx_curr)] = db_curr
    
    return grads_values

def update(params_values, grads_values, nn_architecture, learning_rate):
    for layer_idx, layer in enumerate(nn_architecture):
        params_values["W" + str(layer_idx)] -= learning_rate * grads_values["dW" + str(layer_idx)]        
        params_values["b" + str(layer_idx)] -= learning_rate * grads_values["db" + str(layer_idx)]

    return params_values;

def train(X, Y, nn_architecture, epochs, learning_rate):
    params_values = init_layers(nn_architecture, 2)
    cost_history = []
    accuracy_history = []
    
    for i in range(epochs):
        Y_hat, cashe = full_forward_propagation(X, params_values, nn_architecture)
        cost = get_cost_value(Y_hat, Y)
        cost_history.append(cost)
        accuracy = get_accuracy_value(Y_hat, Y)
        accuracy_history.append(accuracy)
        
        grads_values = full_backward_propagation(Y_hat, Y, cashe, params_values, nn_architecture)
        params_values = update(params_values, grads_values, nn_architecture, learning_rate)
        
    return params_values, cost_history, accuracy_history

train_matrix = train(X_train,y_train,nn_architecture, 2000, 0.1)
test_matrix = train(X_test,y_test,nn_architecture, 3000, 0.1)




#1. Plot the loss curve

In [ ]:
plt.plot([i for i in range(len(train_matrix.cost_history))],(train_matrix.cost_history),color='blue')
plt.plot([i for i in range(len(test_matrix.cost_history))],(test_matrix.cost_history),color= 'red')

#2. Plot the accuracy curve

In [ ]:
plt.plot([i for i in range(len(train_matrix.cost_history))],(train_matrix.accuracy_history),color='blue')
plt.plot([i for i in range(len(test_matrix.cost_history))],(test_matrix.accuracy_history),color= 'red')